In [2]:
import os
import shutil

# Ensure the .kaggle directory exists
os.makedirs("/root/.kaggle", exist_ok=True)

#  Move kaggle.json to the correct location

shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# Set permissions to avoid access issues
os.chmod("/root/.kaggle/kaggle.json", 600)

print(" Kaggle API is now set up correctly!")

 Kaggle API is now set up correctly!


In [3]:
!kaggle datasets download -d adityajn105/flickr8k -p /content --unzip


Dataset URL: https://www.kaggle.com/datasets/adityajn105/flickr8k
License(s): CC0-1.0


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load captions dataset
df = pd.read_csv('/content/captions.txt', delimiter=',')
df.columns = ['image', 'caption']

# Ensure paths are correct
df['image'] = "/content/Images/" + df['image']  # Update image path

# Split 80% train, 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Training Set: {len(train_df)} images")
print(f"Testing Set: {len(test_df)} images")


Training Set: 32364 images
Testing Set: 8091 images


In [6]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load ResNet50 for feature extraction
resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove final classification layer
resnet.eval()

# Load GPT-2 tokenizer & model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

model.eval()  # Set to evaluation mode
model.config.pad_token_id = model.config.eos_token_id  # Avoid warnings


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in al

In [7]:
def extract_features(image_path):
    """Extract image features using ResNet50"""
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        features = resnet(image)

    return features.squeeze().numpy()


#(b) Generate Scene Graph from Caption

In [8]:
import spacy
import networkx as nx

nlp = spacy.load("en_core_web_sm")

def create_scene_graph(caption):
    """Generate scene graph from a given caption"""
    doc = nlp(caption)
    G = nx.DiGraph()

    for token in doc:
        if token.dep_ in ("nsubj", "dobj", "pobj"):  # Only key dependencies
            G.add_edge(token.head.text, token.text)

    return G


In [9]:
def train_model(train_df, num_epochs=5):
    """Train captioning model using 80% dataset"""
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    loss_fn = torch.nn.CrossEntropyLoss()

    model.train()  # Set to training mode

    for epoch in range(num_epochs):
        total_loss = 0

        for _, row in train_df.iterrows():
            image_path = row['image']
            caption = row['caption']

            # Extract image features
            image_features = extract_features(image_path)

            # Generate scene graph
            scene_graph = create_scene_graph(caption)

            # Convert scene graph to text (limit to 5 key relationships)
            scene_text = " ".join([f"{src} {dst}" for src, dst in list(scene_graph.edges)[:5]])

            # Reduce image features to first 10 values for representation
            feature_summary = np.round(image_features[:10], 2).tolist()

            # Prepare input text
            input_text = f"Image features: {feature_summary} Scene: {scene_text} Caption: {caption}"
            input_ids = tokenizer.encode(input_text, return_tensors="pt")

            # Forward pass
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_df):.4f}")

    # Save trained model
    model.save_pretrained("captioning_model")
    tokenizer.save_pretrained("captioning_model")
    print("✅ Training Completed! Model Saved.")


In [10]:
import pandas as pd

# Select only 1% of the test data (which is already 20% of the total dataset)
def sample_1_percent(test_df):
    sample_size = max(1, int(len(test_df) * 0.01))  # Ensure at least one image is tested
    sampled_test_df = test_df.sample(n=sample_size, random_state=42)  # Randomly sample
    print(f"🔹 Testing on {sample_size} images out of {len(test_df)} total test images.\n")
    return sampled_test_df

# Sample 1% from test data
test_sample_df = sample_1_percent(test_df)


🔹 Testing on 80 images out of 8091 total test images.



In [11]:
def generate_caption(image_features, scene_graph, max_new_tokens=30):
    """Generate caption using trained GPT-2 model with proper attention mask and pad token handling."""
    model.eval()  # Set to evaluation mode

    # Convert scene graph to text
    scene_text = " ".join([f"{src} {dst}" for src, dst in list(scene_graph.edges)[:5]])

    # Reduce image features to first 10 values
    feature_summary = np.round(image_features[:10], 2).tolist()

    # Prepare input text
    input_text = f"Image features: {feature_summary} Scene: {scene_text} Caption:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Set pad_token_id and attention_mask
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)  # Ensure attention mask is applied
    model.config.pad_token_id = tokenizer.eos_token_id  # Set pad token ID

    # Generate caption with attention mask
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Fix warning by setting attention mask
        max_new_tokens=max_new_tokens,
        do_sample=True
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [12]:
def test_model(test_sample_df):
    """Test caption generation on 1% of 20% test dataset."""
    for _, row in test_sample_df.iterrows():
        image_path = row['image']
        reference_caption = row['caption']

        # Extract image features
        image_features = extract_features(image_path)

        # Generate scene graph
        scene_graph = create_scene_graph(reference_caption)

        # Generate caption
        generated_caption = generate_caption(image_features, scene_graph)

        print(f"\n📌 Image: {image_path}")
        print(f"✅ Reference Caption: {reference_caption}")
        print(f"🚀 Generated Caption: {generated_caption}")

# Run the test on only 1% of the test data
test_model(test_sample_df)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📌 Image: /content/Images/2174206711_11cb712a8d.jpg
✅ Reference Caption: Two asian girls cheerleading for a sporting event
🚀 Generated Caption: Image features: [0.25999999046325684, 0.44999998807907104, 2.390000104904175, 0.12999999523162842, 1.440000057220459, 0.47999998927116394, 0.07999999821186066, 0.0, 0.3499999940395355, 1.0700000524520874] Scene: cheerleading girls for event Caption: Scene: cheerleading girls for event

RAW Paste Data

Scene: cheerleading girls for event A scene of a cheering cheerleader, cheer

📌 Image: /content/Images/3477672764_7f07657a26.jpg
✅ Reference Caption: A soccer player is about to kick the ball near the goal .
🚀 Generated Caption: Image features: [1.1200000047683716, 0.5, 1.6200000047683716, 0.03999999910593033, 0.5199999809265137, 0.12999999523162842, 0.6000000238418579, 0.3499999940395355, 0.10000000149011612, 1.2799999713897705] Scene: is player kick ball near goal Caption: Frame size: 1.1b Scene: is player kick ball near goal Caption: Frame size

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📌 Image: /content/Images/3667157255_4e66d11dc2.jpg
✅ Reference Caption: An Olympic winner takes home two medals .
🚀 Generated Caption: Image features: [0.47999998927116394, 0.9800000190734863, 1.1799999475479126, 0.0, 1.149999976158142, 0.27000001072883606, 1.3799999952316284, 0.019999999552965164, 0.20999999344348907, 1.75] Scene: takes winner takes medals Caption: [Screenshot] http://i.imgur.com/QKQgJh5.png [/script/screenshots] Source: [URL


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...


📌 Image: /content/Images/133905560_9d012b47f3.jpg
✅ Reference Caption: Two yellow labs are playing in the snow .
🚀 Generated Caption: Image features: [0.4699999988079071, 0.30000001192092896, 0.5600000023841858, 0.029999999329447746, 0.3400000035762787, 0.09000000357627869, 0.03999999910593033, 0.0, 0.8999999761581421, 0.2800000011920929] Scene: playing labs in snow Caption: [0.4699999988079071, 0.30000001192092896, 0.5600000023841858,


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📌 Image: /content/Images/3760400645_3ba51d27f9.jpg
✅ Reference Caption: a girl at a wedding holding some orange flowers
🚀 Generated Caption: Image features: [0.009999999776482582, 0.5400000214576721, 0.3400000035762787, 0.019999999552965164, 0.30000001192092896, 0.33000001311302185, 1.0199999809265137, 0.2199999988079071, 0.27000001072883606, 0.7200000286102295] Scene: at wedding holding flowers Caption: [0.009999999776482582, 0.5400000214576721, 0.3400000035762787, 0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📌 Image: /content/Images/3724487641_d2096f10e5.jpg
✅ Reference Caption: A bunch of little boys in bathing trunks running .
🚀 Generated Caption: Image features: [0.6000000238418579, 1.3200000524520874, 0.20999999344348907, 0.25, 0.30000001192092896, 0.550000011920929, 1.0399999618530273, 0.3799999952316284, 0.029999999329447746, 0.8199999928474426] Scene: of boys in trunks Caption: [0.60000002389413544, 0.5000000238544787834, 0.1900000023941844

📌 Image: /content/Images/2815745115_c8479d560c.jpg
✅ Reference Caption: Lady in a stuffed animal store .
🚀 Generated Caption: Image features: [0.15000000596046448, 0.5400000214576721, 0.949999988079071, 0.029999999329447746, 0.5199999809265137, 0.1599999964237213, 0.4699999988079071, 0.3799999952316284, 0.2199999988079071, 0.75] Scene: in store Caption: [0.100000099493360, -0.03000014006500, -0.049999999061280,


In [13]:
!pip uninstall -y nltk
!pip install --upgrade nltk


Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


In [14]:
import shutil
import os
import nltk

# Delete existing NLTK data folder
nltk_data_path = "/root/nltk_data"
if os.path.exists(nltk_data_path):
    shutil.rmtree(nltk_data_path)  # Delete folder

# Recreate and set path
os.makedirs(nltk_data_path)
nltk.data.path.append(nltk_data_path)

# Redownload required resources
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('omw-1.4', download_dir=nltk_data_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [18]:
import nltk

# Uninstall and reinstall NLTK
!pip uninstall -y nltk
!pip install --upgrade nltk

# Download required NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


Found existing installation: nltk 3.9.1
Uninstalling nltk-3.9.1:
  Successfully uninstalled nltk-3.9.1
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [22]:
!pip install sacrebleu
import sacrebleu

def evaluate_bleu_sacrebleu(test_sample_df):
    """Calculate BLEU score using SacreBLEU instead of NLTK."""
    scores = []

    for _, row in test_sample_df.iterrows():
        reference_caption = row['caption']
        image_features = extract_features(row['image'])
        scene_graph = create_scene_graph(reference_caption)
        generated_caption = generate_caption(image_features, scene_graph)

        # Compute BLEU using sacrebleu
        score = sacrebleu.sentence_bleu(generated_caption, [reference_caption]).score
        scores.append(score)

        print(f"\n✅ Reference Caption: {reference_caption}")
        print(f"🚀 Generated Caption: {generated_caption}")
        print(f"🎯 SacreBLEU Score: {score:.4f}")

    # Calculate average BLEU score
    avg_bleu = sum(scores) / len(scores) if scores else 0
    print(f"\n📊 Average SacreBLEU Score: {avg_bleu:.4f}")

# Run the evaluation
evaluate_bleu_sacrebleu(test_sample_df)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.9 MB/s eta 0:00:00


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✅ Reference Caption: Two asian girls cheerleading for a sporting event
🚀 Generated Caption: Image features: [0.25999999046325684, 0.44999998807907104, 2.390000104904175, 0.12999999523162842, 1.440000057220459, 0.47999998927116394, 0.07999999821186066, 0.0, 0.3499999940395355, 1.0700000524520874] Scene: cheerleading girls for event Caption: - - - Full name: Wahlburgs - - - Year: 18th Century - Language and culture: German - Affiliation: Austria,
🎯 SacreBLEU Score: 0.9011


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✅ Reference Caption: A soccer player is about to kick the ball near the goal .
🚀 Generated Caption: Image features: [1.1200000047683716, 0.5, 1.6200000047683716, 0.03999999910593033, 0.5199999809265137, 0.12999999523162842, 0.6000000238418579, 0.3499999940395355, 0.10000000149011612, 1.2799999713897705] Scene: is player kick ball near goal Caption: Loading Screen... 0.132000000, 0.2720000384039068 0.232000002, 0.304412
🎯 SacreBLEU Score: 2.2241


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✅ Reference Caption: The young basketball player moves into the front court .
🚀 Generated Caption: Image features: [0.15000000596046448, 0.17000000178813934, 0.3700000047683716, 0.07000000029802322, 1.7400000095367432, 0.2800000011920929, 0.9100000262260437, 0.2199999988079071, 0.029999999329447746, 0.30000001192092896] Scene: moves player into court Caption: GECKING GECK (0.0000000134372428), 0.00000000, 1570534468, 2137703906
🎯 SacreBLEU Score: 1.2053


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✅ Reference Caption: A man is backpacking up a grassy hill .
🚀 Generated Caption: Image features: [0.10999999940395355, 0.2800000011920929, 0.029999999329447746, 0.33000001311302185, 0.11999999731779099, 0.4099999964237213, 0.47999998927116394, 0.009999999776482582, 0.029999999329447746, 0.029999999329447746] Scene: backpacking man backpacking hill Caption: photo from Flickr user loya_diaz1 Caption: photo from Flickr user loya_diaz1 Caption: Photo from Flickr user
🎯 SacreBLEU Score: 0.8865


KeyboardInterrupt: 

In [23]:
!pip install jiwer
from jiwer import wer

def evaluate_meteor_jiwer(test_sample_df):
    """Calculate METEOR-like score using Word Error Rate (WER)."""
    scores = []

    for _, row in test_sample_df.iterrows():
        reference_caption = row['caption']
        image_features = extract_features(row['image'])
        scene_graph = create_scene_graph(reference_caption)
        generated_caption = generate_caption(image_features, scene_graph)

        # Compute WER (Lower WER means better match)
        score = 1 - wer(reference_caption, generated_caption)  # Convert WER to similarity
        scores.append(score)

        print(f"\n✅ Reference Caption: {reference_caption}")
        print(f"🚀 Generated Caption: {generated_caption}")
        print(f"🎯 WER-based METEOR Score: {score:.4f}")

    avg_meteor = sum(scores) / len(scores) if scores else 0
    print(f"\n📊 Average WER-based METEOR Score: {avg_meteor:.4f}")

# Run the evaluation
evaluate_meteor_jiwer(test_sample_df)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.9 MB/s eta 0:00:00


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✅ Reference Caption: Two asian girls cheerleading for a sporting event
🚀 Generated Caption: Image features: [0.25999999046325684, 0.44999998807907104, 2.390000104904175, 0.12999999523162842, 1.440000057220459, 0.47999998927116394, 0.07999999821186066, 0.0, 0.3499999940395355, 1.0700000524520874] Scene: cheerleading girls for event Caption: [9] <Cupcakes> 1) I just came here with more money. 2) So when the time comes for the new year,
🎯 WER-based METEOR Score: -3.5000


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✅ Reference Caption: A soccer player is about to kick the ball near the goal .
🚀 Generated Caption: Image features: [1.1200000047683716, 0.5, 1.6200000047683716, 0.03999999910593033, 0.5199999809265137, 0.12999999523162842, 0.6000000238418579, 0.3499999940395355, 0.10000000149011612, 1.2799999713897705] Scene: is player kick ball near goal Caption: is not set: [8:33:19] [09:08:23.0013] [09:07:19.9957
🎯 WER-based METEOR Score: -0.8462


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✅ Reference Caption: An Olympic winner takes home two medals .
🚀 Generated Caption: Image features: [0.47999998927116394, 0.9800000190734863, 1.1799999475479126, 0.0, 1.149999976158142, 0.27000001072883606, 1.3799999952316284, 0.019999999552965164, 0.20999999344348907, 1.75] Scene: takes winner takes medals Caption: Results: M1 Match Results M2 Match Results M3 Match Results M4 Match Results F1 Match Results P/O1 Match Results M5
🎯 WER-based METEOR Score: -3.5000


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



✅ Reference Caption: Two brown dogs runs through the water .
🚀 Generated Caption: Image features: [0.12999999523162842, 0.2800000011920929, 0.30000001192092896, 0.1599999964237213, 0.05999999865889549, 0.4300000071525574, 0.27000001072883606, 0.6399999856948853, 0.4000000059604645, 0.75] Scene: runs dogs through water Caption: Screen shots

RAW Paste Data

Scene #0: Water from a river [0.009700000182405, 0.10
🎯 WER-based METEOR Score: -2.6250


KeyboardInterrupt: 